In [31]:
delays = [1, 0.7, 0.7, 0.5, 0.2, 0.2, 1, 0.8, 0.2, 0.4, 0.5, 0.8]
print("Sum of all delays: %f" % sum(delays))
for ff1_pos in range(len(delays)):
    for ff2_pos in range(ff1_pos + 1, len(delays)):
        start_to_ff1 = sum(delays[0:ff1_pos])
        ff1_to_ff2 = sum(delays[ff1_pos:ff2_pos])
        ff2_to_end = sum(delays[ff2_pos:len(delays)])
        total = start_to_ff1 + ff1_to_ff2 + ff2_to_end
        max_of_delays = max(start_to_ff1, ff1_to_ff2, ff2_to_end)
        if max_of_delays <= 3.0:
            print("ff1: %02d ff2: %02d start_to_ff1: %f ff1_to_ff2: %f ff2_to_end: %f max_of_delays: %f" % 
                  (ff1_pos, ff2_pos, start_to_ff1, ff1_to_ff2, ff2_to_end, max_of_delays))

Sum of all delays: 7.000000
ff1: 02 ff2: 07 start_to_ff1: 1.700000 ff1_to_ff2: 2.600000 ff2_to_end: 2.700000 max_of_delays: 2.700000
ff1: 03 ff2: 07 start_to_ff1: 2.400000 ff1_to_ff2: 1.900000 ff2_to_end: 2.700000 max_of_delays: 2.700000
ff1: 03 ff2: 08 start_to_ff1: 2.400000 ff1_to_ff2: 2.700000 ff2_to_end: 1.900000 max_of_delays: 2.700000
ff1: 03 ff2: 09 start_to_ff1: 2.400000 ff1_to_ff2: 2.900000 ff2_to_end: 1.700000 max_of_delays: 2.900000
ff1: 04 ff2: 07 start_to_ff1: 2.900000 ff1_to_ff2: 1.400000 ff2_to_end: 2.700000 max_of_delays: 2.900000
ff1: 04 ff2: 08 start_to_ff1: 2.900000 ff1_to_ff2: 2.200000 ff2_to_end: 1.900000 max_of_delays: 2.900000
ff1: 04 ff2: 09 start_to_ff1: 2.900000 ff1_to_ff2: 2.400000 ff2_to_end: 1.700000 max_of_delays: 2.900000
ff1: 04 ff2: 10 start_to_ff1: 2.900000 ff1_to_ff2: 2.800000 ff2_to_end: 1.300000 max_of_delays: 2.900000


In [68]:
max_delays = [1, 0.7, 0.7, 0.5, 0.2, 0.2, 1, 0.8, 0.2, 0.4, 0.5, 0.8]
min_delays = [0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1]
t_setup = 0.1
t_hold = 0.0
t_clkq = 0.150
t_dq = 0.150
for l1_pos in range(len(max_delays)):
    for l2_pos in range(l1_pos + 1, len(max_delays)):
        start_to_l1_max = sum(max_delays[0:l1_pos])
        l1_to_l2_max = sum(max_delays[l1_pos:l2_pos])
        l2_to_end_max = sum(max_delays[l2_pos:len(max_delays)])
        
        start_to_l1_min = sum(min_delays[0:l1_pos])
        l1_to_l2_min = sum(min_delays[l1_pos:l2_pos])
        l2_to_end_min = sum(min_delays[l2_pos:len(min_delays)])

        # Compute the ideal max clock period based on longest logic path
        path_delays = [start_to_l1_max, l1_to_l2_max, l2_to_end_max]
        path_delays.sort() # ascending sort
        T_clk = (t_clkq + path_delays[2] + t_setup) / 1.5
        
        # Compute t_borrow,mid and t_borrow,min
        t_borrow_mid = (1.5 * T_clk) - t_dq - path_delays[1]
        t_borrow_min = (1.5 * T_clk) - t_dq - path_delays[0]
        
        # Perform borrowing checks
        borrow_ok = t_borrow_mid + t_borrow_min < T_clk
        if not borrow_ok:
            T_clk = (t_borrow_mid + t_borrow_min) / 0.5
            
        # Check hold time constraints for each path
        min_path_delays = [start_to_l1_min, l1_to_l2_min, l2_to_end_min]
        hold2_ok = 0.5 * T_clk + t_hold < t_clkq + min_path_delays[2]
        hold1_ok = 0.5 * T_clk + t_hold < t_clkq + min_path_delays[1]
        hold0_ok = 0.5 * T_clk + t_hold < t_clkq + min_path_delays[0]
        hold_ok = hold2_ok and hold1_ok and hold0_ok
        
        if T_clk < 2:
            print("l1: %02d l2: %.2d T_clk: %.2f max delays: (%.2f %.2f %.2f) min delays: (%.2f %.2f %.2f) hold_ok: (%d, %d, %d)" 
                  % (l1_pos, l2_pos, T_clk, start_to_l1_max, l1_to_l2_max, l2_to_end_max, 
                     start_to_l1_min, l1_to_l2_min, l2_to_end_min, hold0_ok, hold1_ok, hold2_ok))

l1: 02 l2: 07 T_clk: 1.97 max delays: (1.70 2.60 2.70) min delays: (0.30 0.50 0.60) hold_ok: (0, 0, 0)
l1: 03 l2: 07 T_clk: 1.97 max delays: (2.40 1.90 2.70) min delays: (0.40 0.40 0.60) hold_ok: (0, 0, 0)
l1: 03 l2: 08 T_clk: 1.97 max delays: (2.40 2.70 1.90) min delays: (0.40 0.50 0.50) hold_ok: (0, 0, 0)
